In [160]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

In [161]:
# ランダムシードを設定
torch.manual_seed(42)

class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__() #初期化
        #中間層と出力層を定義
        self.layers = nn.ModuleList([
            nn.Linear(3,6),
            nn.Linear(6,4),
            nn.Linear(4,2)
        ])
        self.output_layer = nn.Linear(2, 1)
        
    def forward(self, x): #順伝播
        for layer in self.layers: #中間層に入れて活性化関数Reluかませる
            x = torch.relu( layer(x) )
        x = self.output_layer(x) #出力層
        x = x #ここで出力層の活性化関数今回は恒等関数
        return x

model = SimpleNN()





In [162]:
df_train = pd.read_csv("purchase.csv")

df_train.head()

,チョコ120円(個),アメ50円(個),ガム150円(個),合計金額(円)
0,6,3,7,1920
1,4,6,9,2130
2,2,6,7,1590
3,4,3,7,1680
4,7,2,5,1690


In [163]:
X_train = df_train.drop(columns=["合計金額(円)"]).values
Y_train = df_train[["合計金額(円)"]].values



In [164]:
#from sklearn.preprocessing import StandardScaler

#scaler_X = StandardScaler()
#scaler_Y = StandardScaler()
#X_train = torch.tensor(scaler_X.fit_transform(X_train), dtype=torch.float32)
#Y_train = torch.tensor(scaler_Y.fit_transform(Y_train), dtype=torch.float32)


In [165]:
X_train = torch.tensor(X_train, dtype=torch.float32)
Y_train = torch.tensor(Y_train, dtype=torch.float32)


In [166]:
# 学習ループ

optimizer = optim.Adam(model.parameters(), lr=0.1) #更新は確率的勾配降下法で
criterion = nn.MSELoss() #損失関数は二乗誤差関数
num_epochs = 1000
for epoch in range(num_epochs):
    # 順伝播
    outputs = model(X_train)
    loss = criterion(outputs, Y_train)

    # 勾配をリセット
    optimizer.zero_grad()

    # 逆伝播
    loss.backward()

    # パラメータの更新
    optimizer.step()
    
    # 10エポックごとに損失を表示
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [10/1000], Loss: 1715990.8750
Epoch [20/1000], Loss: 370827.3125
Epoch [30/1000], Loss: 163076.7500
Epoch [40/1000], Loss: 46081.1562
Epoch [50/1000], Loss: 12074.5156
Epoch [60/1000], Loss: 2102.8286
Epoch [70/1000], Loss: 2281.6265
Epoch [80/1000], Loss: 1912.0775
Epoch [90/1000], Loss: 938.8922
Epoch [100/1000], Loss: 810.5061
Epoch [110/1000], Loss: 739.3097
Epoch [120/1000], Loss: 627.6992
Epoch [130/1000], Loss: 575.6696
Epoch [140/1000], Loss: 515.5178
Epoch [150/1000], Loss: 464.9977
Epoch [160/1000], Loss: 416.2054
Epoch [170/1000], Loss: 371.6796
Epoch [180/1000], Loss: 330.3993
Epoch [190/1000], Loss: 292.5015
Epoch [200/1000], Loss: 257.8708
Epoch [210/1000], Loss: 226.3874
Epoch [220/1000], Loss: 197.9217
Epoch [230/1000], Loss: 172.3180
Epoch [240/1000], Loss: 149.4074
Epoch [250/1000], Loss: 129.0079
Epoch [260/1000], Loss: 110.9350
Epoch [270/1000], Loss: 95.0037
Epoch [280/1000], Loss: 81.0294
Epoch [290/1000], Loss: 68.8398
Epoch [300/1000], Loss: 58.2681
Epoch 

In [167]:
df_test = pd.read_csv("purchase_test.csv")
X_test = df_test.drop(columns=["合計金額(円)"]).values
Y_test = df_test[["合計金額(円)"]].values
X_test = torch.tensor(X_test, dtype=torch.float32)
Y_test = torch.tensor(Y_test, dtype=torch.float32)

In [ ]:
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    test_loss = criterion(test_outputs, Y_test)
    
    # 精度の計算 (例: 平均二乗誤差やR^2スコア)
    mse = test_loss.item()
    print(f'Test Loss (MSE): {mse:.4f}')

tensor(0.3723)
Test Loss (MSE): 0.3723


In [169]:
from sklearn.metrics import r2_score

# R^2スコアの計算
r2 = r2_score(Y_test.numpy(), test_outputs.numpy())
print(f'R^2 Score: {r2:.4f}')


R^2 Score: 1.0000
